In [ ]:
import pandas as pd
from pathlib import Path

### Fleet analysis
This plot provides additional analysis on the fleet composition in the scenarios

In [ ]:
# Inputs and outputs
results_path = Path("../results")
tables_path = Path("tables")

In [ ]:
# Prepare case information
values = [100, 200, 500, 1000]

cases = [
    "baseline_2019", "today_2024", "future_2030",
    "policy_co2_100EUR", "policy_co2_200EUR", "policy_co2_500EUR", "policy_co2_1000EUR", "policy_co2_1500EUR", "policy_co2_2000EUR",
    "policy_thermic_20pct", "policy_thermic_50pct", "policy_thermic_75pct", "policy_thermic_100pct",
    "policy_elez", "policy_electric"
]

df_fleet = []

for case in cases:
    df_partial = pd.read_parquet(results_path / "scenario_solutions/{}.vehicles.parquet".format(case))
    df_partial = df_partial.groupby("vehicle_type").size().reset_index(name = "count")
    df_partial["case"] = case
    df_fleet.append(df_partial)

df_fleet = pd.concat(df_fleet)

In [ ]:
# Prepare fleet composition table
df_table = df_fleet.copy()

df_table = df_table.pivot(
    index = "case", columns = "vehicle_type", values = "count").fillna(0).astype(int)

df_table = df_table[[
    "small_thermic", "small_electric",
    "medium_thermic", "medium_electric",
    "large_thermic", "large_electric"
]]

df_table["sorter"] = [cases.index(c) for c in df_table.index]
df_table = df_table.sort_values(by = "sorter").drop(columns = ["sorter"])

df_table.columns = pd.MultiIndex.from_product([
    ("Small", "Medium", "Large"), ("ICV", "BEV")
])

df_table.index.name = None

def replace_name(name):
    if name == "baseline_2019": return "Baseline 2019"
    if name == "today_2024": return "Today 2024"
    if name == "future_2030": return "Future 2030"
    if name == "policy_elez": return "Electric LEZ"
    if name == "policy_electric": return "100\\% Electric"

    if name.startswith("policy_co2"):
        return "Carbon tax ({} EUR)".format(name.split("_")[-1].replace("EUR", ""))

    if name.startswith("policy_thermic"):
        return "ICV tax ({}\\%)".format(name.split("_")[-1].replace("pct", ""))

    return name

df_table.index = [replace_name(name) for name in df_table.index]

with open(tables_path / "vehicle_counts.tex", "w+") as f:
    table = df_table.to_latex(
        caption = "Fleet composition results across scenarios", 
        label = "tab:fleet_composition", sparsify = False
    )

    table = table.replace("Carbon tax (100 EUR)", "\\midrule\nCarbon tax (100 EUR)")
    table = table.replace("ICV tax (20\\%)", "\\midrule\nICV tax (20\\%)")
    table = table.replace("Electric LEZ", "\\midrule\nElectric LEZ")

    table = table.replace("\\begin{table}", "\\begin{table}\n\\centering")
    
    f.write(table)

df_table